## 2.1 エジプト乗法

乗法は足し算で表すことが可能

In [81]:
import logging
# ログの基本設定
logging.basicConfig(level=logging.DEBUG) # 通常はINFOに設定する
# 処理回数計測用
cnt: int = 0

In [82]:
# 通常の加算。計算はPythonに任せる
def multiply(n: int, a:int) -> int:
  global cnt
  cnt += 1
  return n*a

In [83]:
# 時間を計測する 41*59
cnt = 0
%time multiply(41, 59)
print(f'呼び出し回数: {cnt}')

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.68 µs
呼び出し回数: 1


In [84]:
# 乗算を足し算にして実行する。再帰呼び出しとなるので、スタックを消費する
def multiply0(n: int, a:int) -> int:
  ''' n回の実行：O(n) '''
  global cnt
  cnt += 1
  if n == 1: return a
  return multiply0( n-1, a ) + a

In [85]:
# 時間を計測する 41*59
cnt = 0
%time multiply0(41, 59)
print(f'呼び出し回数: {cnt}')

CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 14.1 µs
呼び出し回数: 41


In [86]:
# 計算の効率化
def odd(n: int) -> bool:
  return (n & 0x01)
def half(n: int) -> int:
  return n >> 1
def multiply1(n: int, a:int) -> int:
  ''' O(log n)までに減る '''
  global cnt
  cnt += 1
  if n == 1: return a
  result: int = multiply1( half(n), a + a )
  if odd(n): result += a
  return result

In [87]:
# 時間を計測する 41*59
cnt = 0
%time result = multiply1(41, 59)
print(f'計算結果： {result}')
print(f'呼び出し回数: {cnt}')

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 11.9 µs
計算結果： 2419
呼び出し回数: 6


### 加法連鎖

上記、エジプト乗法(ロシア農民のアルゴリズム)が最良とも限らない場合がある。

関数呼び出し回数ではなく、加算回数に着目する。

In [88]:
add_cnt = 0
def multiply1(n: int, a:int) -> int:
  ''' O(log n)までに減る '''
  global cnt, add_cnt
  cnt += 1
  if n == 1: return a
  add_cnt += 1
  result: int = multiply1( half(n), a + a )
  if odd(n): result += a
  if odd(n): add_cnt +=1
  return result

In [89]:
# 時間を計測する 15*49
cnt = 0
add_cnt = 0
%time result = multiply1(15, 49)
print(f'計算結果： {result}')
print(f'呼び出し回数: {cnt}')
print(f'加算回数: {add_cnt}')

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11 µs
計算結果： 735
呼び出し回数: 4
加算回数: 6


In [90]:
# 15を加法連鎖で計算する場合
def multiply_by_15(a: int) -> int:
  global cnt, add_cnt
  cnt += 1
  add_cnt += 2
  b: int  = ( a + a ) + a # b == 3*a
  add_cnt += 1
  c: int  = ( b + b )     # c == 2*b == 6a
  add_cnt += 2
  return ( c + c ) + b    # 2*c + b => 12a + 3a => 15a

In [91]:
# 時間を計測する 15*49
cnt = 0
add_cnt = 0
%time result = multiply_by_15(49)
print(f'計算結果： {result}')
print(f'呼び出し回数: {cnt}')
print(f'加算回数: {add_cnt}')

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.3 µs
計算結果： 735
呼び出し回数: 1
加算回数: 5


#### 問題2-1

n < 100 の最適な加法連鎖を求めよ。

In [92]:
# キューについての理解メモ
from collections import deque
queue = deque()
# 初期化時点ではキューには何も含まれていない
if not queue:
    print(queue)
    print("queue is empty")

# キューにエントリを追加する(タプル型データ)
queue.append(([1], set([1])))
if not queue:
    print("queue is empty")
# キューの中身を覗いてみる
print(queue)
# タイプを確認してみる
print(type(queue))

deque([])
queue is empty
deque([([1], {1})])
<class 'collections.deque'>


In [93]:
# リストの連結
print([1]+[2])
# set集合の演算(和を取る)
print( {1} | {2} )

[1, 2]
{1, 2}


In [94]:
'''
Microsoft Copilotに質問して得られた加法連鎖のコード
'''
from collections import deque

def shortest_addition_chain(n):
    """n に対する最短加法連鎖を求める"""
    queue = deque()
    # nqueue（追加）
    queue.append(([1], set([1]))) # キューの末尾にタプル([1], set(1))を追加する

    while queue: # キューがから出ない限り継続する
        # Dequeue（取り出し）
        chain, seen = queue.popleft() # キューの先頭から取り出す(タプルは分解する)
        # chainの最後を取得する
        last = chain[-1]

        # 判定処理： chainの最後が n に一致する場合、chainを返す
        if last == n:
            return chain

        # chainのリストを反転させてイテレートする
        for i in reversed(chain):
            new_val = last + i # chainの末尾に1を加えた値をnew_valに設定する
            if new_val <= n and new_val not in seen: # n以下かつseenに含まれていないこと
                new_chain = chain + [new_val]
                new_seen = seen | {new_val}
                queue.append((new_chain, new_seen))
                #print(queue)

In [95]:
i = 4
chain = shortest_addition_chain(i)
print(chain)

[1, 2, 4]


In [96]:
'''
Microsoft Copilotに質問して得られた加法連鎖のコードを見直し
加法連鎖の候補を見つけた段階でchainを返すことで処理が改善される
'''
from collections import deque

def shortest_addition_chain(n):
    """n に対する最短加法連鎖を求める"""
    queue = deque()
    # nqueue（追加）
    queue.append(([1], set([1]))) # キューの末尾にタプル([1], set(1))を追加する

    # 判定処理： chainの最後が n に一致する場合、chainを返す
    if 1 == n:
       return [1]

    while queue: # キューが空にならない限り継続する
        # Dequeue（取り出し）
        chain, seen = queue.popleft() # キューの先頭から取り出す(タプルは分解する)
        # chainの最後を取得する
        last = chain[-1]

        # chainのリストを反転させてイテレートする
        for i in reversed(chain):
            new_val = last + i # chainの末尾に1を加えた値をnew_valに設定する
            if new_val <= n and new_val not in seen: # n以下かつseenに含まれていないこと
                new_chain = chain + [new_val]
                new_seen = seen | {new_val}
                queue.append((new_chain, new_seen))
                # 判定を見つけた段階で実施する(25秒前後の処理が改善するか？)
                if new_val == n:
                    return new_chain

In [97]:
tables = []
# 利用例
def generate_addition_chain_table(limit):
    """1〜limitまでの加法連鎖テーブルを作成"""
    table = {}
    for i in range(1, limit + 1):
        chain = shortest_addition_chain(i)
        table[i] = chain
    return table

# 実行例
limit = 99
%time table = generate_addition_chain_table(limit)
global tables
tables = table

# 表示
for k, v in table.items():
    print(f"{k}: {' → '.join(map(str, v))}（長さ: {len(v)}）")

CPU times: user 3.13 s, sys: 151 ms, total: 3.28 s
Wall time: 3.28 s
1: 1（長さ: 1）
2: 1 → 2（長さ: 2）
3: 1 → 2 → 3（長さ: 3）
4: 1 → 2 → 4（長さ: 3）
5: 1 → 2 → 4 → 5（長さ: 4）
6: 1 → 2 → 4 → 6（長さ: 4）
7: 1 → 2 → 4 → 6 → 7（長さ: 5）
8: 1 → 2 → 4 → 8（長さ: 4）
9: 1 → 2 → 4 → 8 → 9（長さ: 5）
10: 1 → 2 → 4 → 8 → 10（長さ: 5）
11: 1 → 2 → 4 → 8 → 10 → 11（長さ: 6）
12: 1 → 2 → 4 → 8 → 12（長さ: 5）
13: 1 → 2 → 4 → 8 → 12 → 13（長さ: 6）
14: 1 → 2 → 4 → 8 → 12 → 14（長さ: 6）
15: 1 → 2 → 4 → 5 → 10 → 15（長さ: 6）
16: 1 → 2 → 4 → 8 → 16（長さ: 5）
17: 1 → 2 → 4 → 8 → 16 → 17（長さ: 6）
18: 1 → 2 → 4 → 8 → 16 → 18（長さ: 6）
19: 1 → 2 → 4 → 8 → 16 → 18 → 19（長さ: 7）
20: 1 → 2 → 4 → 8 → 16 → 20（長さ: 6）
21: 1 → 2 → 4 → 8 → 16 → 20 → 21（長さ: 7）
22: 1 → 2 → 4 → 8 → 16 → 20 → 22（長さ: 7）
23: 1 → 2 → 4 → 5 → 9 → 18 → 23（長さ: 7）
24: 1 → 2 → 4 → 8 → 16 → 24（長さ: 6）
25: 1 → 2 → 4 → 8 → 16 → 24 → 25（長さ: 7）
26: 1 → 2 → 4 → 8 → 16 → 24 → 26（長さ: 7）
27: 1 → 2 → 4 → 8 → 9 → 18 → 27（長さ: 7）
28: 1 → 2 → 4 → 8 → 16 → 24 → 28（長さ: 7）
29: 1 → 2 → 4 → 8 → 16 → 24 → 28 → 29（長さ: 8）
30

In [98]:
# 15の加法連鎖のchainは１つではない
print(tables[15])

[1, 2, 4, 5, 10, 15]


In [99]:
def multiply_using_addition_chain(base: int, multiplier: int, chain_table: dict[int, list[int]]) -> int:
    if multiplier not in chain_table:
        raise ValueError(f"{multiplier} の加法連鎖がテーブルに存在しません")

    chain = chain_table[multiplier]
    values = {1: base}  # 初期値

    # 1から開始してchainリスト個数分ループする
    for i in range(1, len(chain)):
        # ターゲットにchainリスト設定する
        target = chain[i]
        for j in range(i):
            for k in range(j, i):
                if chain[j] + chain[k] == target:
                    values[target] = values[chain[j]] + values[chain[k]]
                    break
            if target in values:
                break

    return values[multiplier]

In [100]:
result = multiply_using_addition_chain(base=15, multiplier=49, chain_table=tables)
print(f"15 × 49 = {result}")  # 出力: 735

15 × 49 = 735


In [101]:
n = 41
a = 59
%time result = multiply_using_addition_chain(base=n, multiplier=a, chain_table=tables)
print(f"{n} × {a} = {result}")  # 出力: 2419

CPU times: user 18 µs, sys: 2 µs, total: 20 µs
Wall time: 22.2 µs
41 × 59 = 2419


In [102]:
tbl = [1, 2, 4, 5, 10, 15]
def muliply_by_15_2(a: int) -> int:
  b = a + a
  c = b + b + a
  return c + c + c
%time result = muliply_by_15_2(49)
print(result)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs
735


In [103]:
print(tables[15])

[1, 2, 4, 5, 10, 15]


In [104]:
# tbl = [1, 2, 4, 5, 10, 15] を分解する
tbl = [1, 2, 4, 5, 10, 15]
def get_multiply_formula(n: int) -> dict:
  tbl = tables[n].copy()
  last = tbl[-1]
  values = {}
  tbl.reverse()
  for i in range(1, len(tbl)):
    # tbl[i]を導出する
    print(f'{last}を{tbl[i]}以下の値から導出する')
    for j in range(i, len(tbl)):
      if tbl[i] + tbl[j] == last:
        # 組み合わせが見つかった
        print(f'    {last}は{tbl[i]}と{tbl[j]}により導き出せた')
        # 次に導出するのは、tbl[i]
        values[last] = (tbl[i], tbl[j])
        last = tbl[i]
        break
  return values
get_multiply_formula(41)


41を40以下の値から導出する
    41は40と1により導き出せた
40を32以下の値から導出する
    40は32と8により導き出せた
32を16以下の値から導出する
    32は16と16により導き出せた
16を8以下の値から導出する
    16は8と8により導き出せた
8を4以下の値から導出する
    8は4と4により導き出せた
4を2以下の値から導出する
    4は2と2により導き出せた
2を1以下の値から導出する
    2は1と1により導き出せた


{41: (40, 1),
 40: (32, 8),
 32: (16, 16),
 16: (8, 8),
 8: (4, 4),
 4: (2, 2),
 2: (1, 1)}

In [105]:
# 上記方法で導出した加法連鎖のチェインを用いた関数を返す
from typing import Callable
def generate_function(chain: dict[int, tuple[int, int]]) -> Callable[[int], int]:
    def func(a: int) -> int:
        values = {1: a}
        for k in sorted(chain.keys()):
            x, y = chain[k]
            values[k] = values[x] + values[y]
        return values[max(chain.keys())]
    return func

In [106]:
# 加法連鎖を用いた計算
# 加法連鎖テーブルから計算に使う加法連鎖のチェインを取得する
%time dict_chain = get_multiply_formula(15)
%time chain_func = generate_function(dict_chain)
%time print(chain_func(49))


15を10以下の値から導出する
    15は10と5により導き出せた
10を5以下の値から導出する
    10は5と5により導き出せた
5を4以下の値から導出する
    5は4と1により導き出せた
4を2以下の値から導出する
    4は2と2により導き出せた
2を1以下の値から導出する
    2は1と1により導き出せた
CPU times: user 112 µs, sys: 10 µs, total: 122 µs
Wall time: 105 µs
CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.68 µs
735
CPU times: user 15 µs, sys: 1 µs, total: 16 µs
Wall time: 17.9 µs
